#### We need to install module future, not importing from \_\_future\_\_

In [ ]:
#from future.utils import PY3
import future
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import pandas as pd
import numpy as np
import os
import pprint
#from IPython.display import display
import IPython.display as disp
display = disp.display
import matplotlib.pyplot as plt
import scipy.stats as stats
zscore, describe = stats.mstats.zscore, stats.describe
import warnings
import datetime
dt, td = datetime.datetime, datetime.timedelta
import imp

%matplotlib inline

In [ ]:
import ca_lib as la
imp.reload(la)

## Load files

In [ ]:
# Display database folders
display(os.listdir('../_share/Losonczi/'))

# Select animal
#animal = 'msa1215_1'; FPS = 30
#animal = 'msa0216_4'; FPS = 8
animal = 'msa0316_1'; FPS = 8
#animal = 'msa0316_3'; FPS = 8
#animal = 'msa0316ag_1'; FPS = 8

# List dir
mydir = os.path.join('../_share/Losonczi',animal)
os.listdir(mydir)

In [ ]:
# Available trials and ROIs
data = la.load_files(mydir)
print (data.raw.shape, '\n', data.trials, '\n', data.rois)

In [ ]:
# Post-Learning may repeat session_num therefore an additional index,
# day_num is created. See msa0316_1.
# It seems though that Pre-Learning and Learning treats session_num as documented.
display(data.experiment_traits.head())
display(data.experiment_traits[data.experiment_traits['day_leap']])

## Experiment protocol configurations

In [ ]:
et = data.experiment_traits.copy()
et = la.df_epoch(et.groupby(['learning_epoch','context','puffed','port']).size().to_frame(name='count'))
#et.to_clipboard()
disp.display(disp.HTML('<font color="red">ATTENTION, </font>for later conformity we store columns in a <b>different order</b>!!!'))
display(la.df_epoch(et))

et = data.experiment_traits.copy()
etc= et.groupby(['context','port','puffed']).size().to_frame(name='count')
et = la.df_epoch(et.groupby(['learning_epoch','context','port','puffed']).size().to_frame(name='count'))

## Prepare data

In [ ]:
df_data = data.filtered
df_raw = data.raw

In [ ]:
# See how many ROIs are available for which frames

avail_sum = (~data.filtered.isnull()).sum() / len(data.trials)
plt.plot(avail_sum)
plt.xlabel('Camera frame within experiment')
plt.ylabel('Available ROIs on average')

In [ ]:
# See which ROI is available in which trial and for how many frames

avail = ((~data.filtered.isnull()).sum(axis=1)).to_frame('nFrames').unstack()

print(avail.shape)
display(avail.head())
display(avail.tail())

In [ ]:
# Create boolean DataFrame which ROI is spiking in which camera frame

# create empty structure for cumsum
df_template = pd.DataFrame(data=0,index=data.mirow,columns=data.icol)
df_spike = df_template.copy()

# select spike data
spikes = data.transients.loc[data.transients['in_motion_period']==False,['start_frame','stop_frame']]
spikes['count']=1

# fill in spike start and stop points
sp = spikes[['start_frame','count']].pivot(columns='start_frame').fillna(0)
df_spike = df_spike.add(sp['count'], fill_value=0)
sp = spikes[['stop_frame','count']].pivot(columns='stop_frame').fillna(0)
df_spike = df_spike.add(-sp['count'], fill_value=0)

# cumulate, converion to int is not adviced if using NaNs
df_spike = df_spike.cumsum(axis=1).astype(int)

print('table shape', df_spike.shape, 'active frames*ROIs', df_spike.sum().sum())
display(df_spike.head(25))
display(df_spike.tail())

In [ ]:
# Create boolean DataFrame whether licking happens in camera frame

# Check for valid data and calculate their frame
print('All entries', data.behavior.shape)
df_lick = data.behavior[data.behavior.loc[:,'stop_time']>data.behavior.loc[:,'start_time']].copy()
print('Valid licks', df_lick.shape)
df_lick['mid_frame'] = (FPS*(df_lick['start_time']+df_lick['stop_time'])/2).apply(np.round).astype(int)
display(df_lick.head())
display(df_lick.tail())
# Convert to a DataFrame like df_data or df_raw, this eventually skips multiple licks in one camera frame
df_lick = df_lick.reset_index().rename(columns={'index':'time'})
df_lick = df_lick.drop_duplicates(['time','mid_frame']).pivot(index='time', columns='mid_frame')
display(df_lick.head())
df_lick = pd.DataFrame(index=pd.Index([],name='time'),columns=data.icol).append(
    ~df_lick['lick_idx'].isnull()).fillna(0).astype(int)
display(df_lick.head())
# Number of remaining licks
print('Remaining licks',df_lick.sum().sum())
# Smoothen
from scipy.ndimage.filters import gaussian_filter
df_lick = df_lick.apply(lambda x: gaussian_filter(x.astype(float), sigma=2), axis=1, raw=True)
display(df_lick.head())

## z-scoring

In [ ]:
z_spike = la.pd_zscore_by_roi(df_spike, FPS, -2*FPS, axis=1)
z_data = la.pd_zscore_by_roi(df_data, FPS, -2*FPS, axis=1)
z_raw = la.pd_zscore_by_roi(df_raw, FPS, -2*FPS, axis=1)
z_lick = la.pd_zscore_clip(df_lick, FPS, -2*FPS, axis=1)

z_data = z_data.sort_index()
z_raw = z_raw.sort_index()

# Plot

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

class helpmultipage(object):
    def __init__(self, filename):
        self.filename = filename
        self.isopen = False
        self.open()
        
    def __del__(self):
        self.close()
        
    def savefig(self, dpi=None):
        if self.isopen:
            self.pp.savefig(dpi=dpi)

    def open(self):
        if (~self.isopen) and len(self.filename):
            self.pp = PdfPages(self.filename)
            self.isopen = True
        
    def close(self):
        if self.isopen:
            self.pp.close()
        self.isopen = False

#### Explanatory figure

In [ ]:
pp = helpmultipage('explanatory.pdf')

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
center = FPS * (la.events[:-1]+la.events[1:]) /2
left = FPS * la.events
width = FPS * (la.events[1:]-la.events[:-1])
vcenter = 0.0
vstart = -0.5

def label90(x,y,text):
    ax.text(x, y, text, ha="center", va="center", family='sans-serif', size=14, rotation=90)

fig, (empty, ax) = plt.subplots(2,1,figsize=(6,8))
fig.tight_layout(pad=3)
empty.axis('off')
#ax = fig.gca()
fig.suptitle('Explanatory figure')
ax.set_xlabel('Camera frame')
ax.set_ylabel('z-scored activity')
ax.set_ylim(vstart,vstart+1)
ax.plot(z_spike.mean(axis=0)+0.00, label="(CategoryA, True): #trials", c=(1,1,0))
ax.plot(z_spike.mean(axis=0)+0.02, label="(CategoryB, True): #trials", c=(.5,1,.5))
ax.plot(-z_spike.mean(axis=0)+0.00, label="(CategoryA, False): #trials", c=(1,.8,1))
ax.plot(-z_spike.mean(axis=0)+0.02, label="(CategoryB, False): #trials", c=(.5,1,1))
patches = []
# mark delay
label90(center[0], vcenter, 'excitation by\nshowing water')
# mark CS
rect = mpatches.Rectangle((left[1],vstart), width[1], 1, ec="none")
patches.append(rect)
label90(center[1], vcenter, 'CS± if tone\n"Baseline" otherwise')
# mark delay
label90(center[2], vcenter, 'delay')
# mark UC
rect = mpatches.Rectangle((left[3],vstart), width[3], 1, ec="none")
patches.append(rect)
label90(center[3], vcenter, 'UC if any')
# mark water
ax.text((left[0]+left[3])/2, vstart, "water source present\niff allowed to lick",
        ha="center", va="bottom", family='sans-serif', size=14, bbox=dict(boxstyle="DArrow", pad=0.0, fc='c'))

for i in range(0,len(la.events)):
    ax.axvline(x=la.events[i]*FPS, ymin=0.0, ymax = 1.0, linewidth=1, color='k')
colors = np.linspace(0, 1, len(patches))
collection = PatchCollection(patches, cmap=plt.cm.hsv, alpha=0.1)
collection.set_array(np.array(colors))
ax.add_collection(collection)

leg = ax.legend(loc='lower center', title="Category name, Condition name",
               bbox_to_anchor=(0.5, 1.1))
leg.get_title().set_fontsize('large')
leg.get_title().set_fontweight('bold')
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    fig.show()
pp.savefig()

In [ ]:
pp.close()

In [ ]:
pp = helpmultipage(animal+'_pop.pdf')

In [ ]:
la.plot_data(df_spike, df_data, df_lick, data.experiment_traits, FPS)
pp.savefig()

## Z-scored spiking
Spiking is "True" in the [intervals) given in transients_data.hc5

In [ ]:
bsections = np.arange(0,60,5)*FPS
bcenters = (bsections[1:]+bsections[:-1])/2
#mybfun = lambda x: la.func_over_intervals(np.nanmean, bsections, np.array(x))
mybfun = pd.DataFrame.mean

zb_spike = la.pd_aggr_col(z_spike, mybfun, bsections, bcenters.astype(str))
zb_data = la.pd_aggr_col(z_data, mybfun, bsections, bcenters.astype(str))
zb_raw = la.pd_aggr_col(z_raw, mybfun, bsections, bcenters.astype(str))
zb_lick = la.pd_aggr_col(z_lick, mybfun, bsections, bcenters.astype(str))
b_lick = la.pd_aggr_col(df_lick, mybfun, bsections, bcenters.astype(str))

In [ ]:
asections = np.append(la.events,[60])*FPS
acenters = (asections[1:]+asections[:-1])/2
#myafun = lambda x: la.func_over_intervals(np.nanmean, asections, np.array(x))
myafun = pd.DataFrame.mean

za_spike = la.pd_aggr_col(z_spike, myafun, asections, acenters.astype(str))
za_data = la.pd_aggr_col(z_data, myafun, asections, acenters.astype(str))
za_raw = la.pd_aggr_col(z_raw, myafun, asections, acenters.astype(str))
za_lick = la.pd_aggr_col(z_lick, myafun, asections, acenters.astype(str))
a_lick = la.pd_aggr_col(df_lick, myafun, asections, acenters.astype(str))

### Single criterion
* interestingly population activity is high both for puffed and port during the UC session, to be checked in the cross-correlations

In [ ]:
grp = [['context'],['learning_epoch'],['port'],['puffed']]
la.plot_data(z_spike, z_data, df_lick, data.experiment_traits, FPS, grp, title='Population activity')
pp.savefig()
la.plot_data(zb_spike, zb_data, b_lick, data.experiment_traits, FPS, grp, title='Population activity binned', div=bcenters)
pp.savefig()
la.plot_data(za_spike, za_data, a_lick, data.experiment_traits, FPS, grp, title='Population activity averaged over events', div=acenters)
pp.savefig()

### Two criteria
* selecting (port and puffed) makes clear that only the airpuffing correlates with UC (they are the same) and not port
* selecting (epoch and puffed) shows that activity during UC decreases in the post-learning period
* selecting (context and puffed) shows that CS+ alone does not involve higher population activity

### Three criteria
* comments

### All criteria
* There is no increased population activity for CS+ without puffing. (For mouse 0216_4 the 1 trial with port displays increase during the trace period - why?)
* During learning mouse 0216_4 shows incresed activity during the UC phase for CS-

In [ ]:
grp = [['context','port','puffed']]
la.plot_epochs(z_spike, z_data, df_lick, data.experiment_traits, etc, FPS, grp, title='Population activity')
pp.savefig()
la.plot_epochs(zb_spike, zb_data, b_lick, data.experiment_traits, etc, FPS, grp, title='Population activity binned', div=bcenters)
pp.savefig()
la.plot_epochs(za_spike, za_data, a_lick, data.experiment_traits, etc, FPS, grp, title='Population activity averaged over events', div=acenters)
pp.savefig()

### Activities conditional on epoch

In [ ]:
def plot_by_epoch(epoch):
    experiment_c = data.experiment_traits[data.experiment_traits.loc[:,'learning_epoch']==epoch]
    spike_c = z_spike.reindex(experiment_c.index, level='time')
    data_c = z_data.reindex(experiment_c.index, level='time')
    raw_c = z_raw.reindex(experiment_c.index, level='time')
    lick_c = df_lick.reindex(experiment_c.index)
    print (experiment_c.shape, z_spike.shape)
    spike_ca = la.pd_aggr_col(spike_c, myafun, asections, acenters.astype(str))
    data_ca = la.pd_aggr_col(data_c, myafun, asections, acenters.astype(str))
    raw_ca = la.pd_aggr_col(raw_c, myafun, asections, acenters.astype(str))
    lick_ca = la.pd_aggr_col(lick_c, myafun, asections, acenters.astype(str))
    print (spike_c.shape, spike_ca.shape)

    grp = [['context','port'],['context','puffed'],['port','puffed']]
    la.plot_data(spike_c, data_c, lick_c, data.experiment_traits, FPS, grp, title=epoch)
    pp.savefig()
    la.plot_data(spike_ca, data_ca, lick_ca, data.experiment_traits, FPS, grp, title=epoch+' averaged over events', div=acenters)
    pp.savefig()

#### Pre-learning

In [ ]:
plot_by_epoch('Pre-Learning')

#### Learning

In [ ]:
plot_by_epoch('Learning')

#### Post-Learning

In [ ]:
plot_by_epoch('Post-Learning')

In [ ]:
pp.close()

## Individual ROIs
* since there are many of them, save figure to pdf
* THIS WILL <font color="red">TAKE A WHILE</font>, consider testing with a small range

In [ ]:
def plot_roi(df_spike, df_data, filaname, grp, title_template, by_epoch=False, div=None):
    pp = PdfPages(filaname)
    for i in range(0,len(data.rois)):
        spike_c = df_spike.loc[(slice(None),data.rois[i]),:]
        data_c = df_data.loc[(slice(None),data.rois[i]),:]
        #raw_c = df_raw.loc[(slice(None),data.rois[i]),:]
        if by_epoch:
            fig = la.plot_epochs(spike_c, data_c, None, data.experiment_traits, etc, FPS, grp, title=title_template%(i,data.rois[i]), div=div)
        else:
            fig = la.plot_data(spike_c, data_c, None, data.experiment_traits, FPS, grp, title=title_template%(i,data.rois[i]), div=div)
        pp.savefig()
        plt.close(fig)
    pp.close()

raise ValueError("You don't want to run this automaticly")

In [ ]:
import ca_lib as la
imp.reload(la)

In [ ]:
plot_roi(df_spike, df_data, animal+'_roi1crit.pdf',[['context'],['learning_epoch'],['port'],['puffed']],'ROI %d:\n%s')

In [ ]:
plot_roi(df_spike, df_data, animal+'_roiAcrit.pdf',[['context','port','puffed']],'ROI %d:\n%s',True)

### Averaging over intervals

#### Intervals aligned to events

In [ ]:
a_spike = la.pd_aggr_col(df_spike, myafun, asections, acenters.astype(str))
a_data = la.pd_aggr_col(df_data, myafun, asections, acenters.astype(str))
a_raw = la.pd_aggr_col(df_raw, myafun, asections, acenters.astype(str))
a_lick = la.pd_aggr_col(df_lick, myafun, asections, acenters.astype(str))

In [ ]:
a_data = a_data.sort_index()
a_raw = a_raw.sort_index()

In [ ]:
plot_roi(a_spike, a_data, animal+'_avg1crit.pdf',[['context'],['learning_epoch'],['port'],['puffed']],'ROI %d:\n%s',div=acenters)

In [ ]:
plot_roi(a_spike, a_data, animal+'_avgAcrit.pdf',[['context','port','puffed']],'ROI %d:\n%s',True,div=acenters)

#### Averaging over bins

In [ ]:
b_spike = la.pd_aggr_col(df_spike, mybfun, bsections, bcenters.astype(str))
b_data = la.pd_aggr_col(df_data, mybfun, bsections, bcenters.astype(str))
b_raw = la.pd_aggr_col(df_raw, mybfun, bsections, bcenters.astype(str))
b_lick = la.pd_aggr_col(df_lick, mybfun, bsections, bcenters.astype(str))

In [ ]:
b_data = b_data.sort_index()
b_raw = b_raw.sort_index()

In [ ]:
plot_roi(b_spike, b_data, animal+'_bin1crit.pdf',[['context'],['learning_epoch'],['port'],['puffed']],'ROI %d:\n%s',div=bcenters)

In [ ]:
plot_roi(a_spike, a_data, animal+'_binAcrit.pdf',[['context','port','puffed']],'ROI %d:\n%s',True,div=bcenters)

## Correlations

In [ ]:
# Convert to ordinal, here we use indices that way
et1 = experiment_traits.copy().drop('time', axis=1)
et1[['session_num', 'day_num']] = et1[['session_num', 'day_num']].astype(int)
ord1 = z_data.reindex(df_template.index, df_template.columns)
ord1.columns = pd.Index(ord1.columns.values.astype(int), name=ord1.columns.name)

# Combine information
ord1 = ord1.join(et1, how='inner').reset_index().drop('time', axis=1).set_index(['roi_id','learning_epoch','context','port','puffed','session_num']).sort_index()
ord1.columns.name='Spike'
print(ord1.shape)
display(ord1.head())

# Search for days that contain experiments with same traits and session_num
# These entries would jeopardize unstacking
et2 = et1.reset_index(drop=True).set_index(['learning_epoch','context','port','puffed','session_num']).sort_index()
second_occur = et2.index.duplicated()
set1 = et2.loc[second_occur,'day_num'].unique()
all_occur = et2.index.get_duplicates()
set_all = et2.loc[all_occur,'day_num'].unique()
set2 = np.array(list(set(set_all)-set(set1)))
print(set1,set2)

# Filter out second occurrences stored in set2
if len(set2):
    ord1 = ord1[ord1.loc[:,'day_num'].apply(lambda x: x not in set2)]
print(ord1.shape)

# Reshape for correlation analysis
# some value get converted to float to be able to hold nan-s
comp = ord1['day_num'].unstack().sort_index(axis=1)
ord1 = ord1.drop(['day_num'], axis=1).unstack()
ord1 = ord1.reset_index().set_index(['learning_epoch','context','port','puffed','roi_id']).sort_index()
display(comp.head())
display(ord1.head(10))

In [ ]:
# Find the pre-learning structure, without airpuff
key_ref = ('Pre-Learning','CS+',True,False)
time_ref = np.array([15, 40])
col_ref = slice(int(time_ref[0]*FPS),int(time_ref[1]*FPS))
sel = ord1.loc[key_ref+(slice(None),),col_ref]
print(sel.shape)

# Correlate
corr_df = sel.T.corr()
corr_np = corr_df.fillna(0).values

# Discard invalid series
keep = (np.diag(corr_np) == 1.0)
corr_np = corr_np[keep,:][:,keep]

# Show
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].matshow(corr_df.values)
ax[1].matshow(corr_np)

In [ ]:
pp = helpmultipage(animal+'_correl.pdf')

In [ ]:
# Define an ordering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform, pdist
sq_dist = squareform(1.0-corr_np)
corr_link = linkage(sq_dist, 'average')
fig, ax = plt.subplots(1,2, figsize=(18,8))
fig.suptitle('Reference is presented here: '+(', '.join(np.array(key_ref)))+
            ' and time '+('..'.join(time_ref.astype(str)))+'s')
labels = sel.index.get_level_values(4).to_series().reset_index(drop=True)[keep]
dendo = dendrogram(corr_link, ax=ax[1], labels=labels.values, leaf_font_size=2.5, orientation='left')
ax[1].set_title('Distance of firing patterns')
corr_order = dendo['leaves']
# Show reordered
cax = ax[0].matshow(corr_np[corr_order,:][:,corr_order], origin='lower', vmin=-0.8, vmax=1)
ax[0].xaxis.set_ticks_position('bottom')
ax[0].set_title('Ordered correlation matrix', y=1.0)
fig.colorbar(cax)
pp.savefig(dpi=600)

In [ ]:
num_plots = len(et.index)
num_rows = int(np.ceil(num_plots/3.0))
fig, ax = plt.subplots(num_rows,3, figsize=(12,4.5*num_rows))
fig.suptitle('Correlation structure under different conditions: learning_epoch, context, port, puffed\n'+
             '(small number of trials might lead to larger percieved correlation)')
ax = np.ravel(ax)
mx = {}
ds = pd.DataFrame(columns=et.index)

for idx in range(0,num_plots):
    # Find the pre-learning structure
    key = et.index[idx]
    sel = ord1.loc[key+(slice(None),),col_ref]
    print(key,ord1.shape,sel.shape)
    
    # Correlate
    corr_tmp = sel.T.corr()
    corr_tmp = corr_tmp.fillna(0).values

    # Discard invalid series
    #if len(corr_tmp):
    corr_tmp = corr_tmp[keep,:][:,keep][corr_order,:][:,corr_order]
    cax = ax[idx].matshow(corr_tmp, origin='lower', vmin=-0.8, vmax=1)
    ax[idx].xaxis.set_ticks_position('bottom')
        
    mx[et.index[idx]] = corr_tmp
    ds[et.index[idx]] = np.ravel(corr_tmp+np.diag(np.nan*np.diag(corr_tmp)))
    ax[idx].set_title('%s: %d'%(et.index[idx],et.ix[idx]))
pp.savefig(dpi=600)

In [ ]:
fig, ax = plt.subplots(num_rows,3, figsize=(12,4.5*num_rows))
fig.suptitle('Distribution of the above correlation coefficients\n(diagonals excluded)')
ax = np.ravel(ax)

for idx in range(0,num_plots):
    corr_tmp = mx[et.index[idx]]
    corr_tmp = corr_tmp+np.diag(np.nan*np.diag(corr_tmp))
    ax[idx].hist(np.ravel(corr_tmp),range=(-1,1),bins=20)
    ax[idx].set_yscale('log')
    ax[idx].set_title('%s: %d'%(et.index[idx],et.ix[idx]))
    

In [ ]:
#help(pd.tools.plotting.table)
# FIXME index column toooo wide
fig, ax = plt.subplots(1,1)
fig.suptitle('Statistics on the correlation coefficients')
ax.axis('off')
ax.set_position([.5, 0.2, 0.5, 0.6])
a = df_epoch(np.round(ds.describe(),4).T)
cw = np.ones((len(a.columns),))
t = pd.tools.plotting.table(ax, a, loc='upper right', fontsize=12, colWidths=cw/np.sum(cw))
pp.savefig()
plt.close(fig)
a

In [ ]:
#help(pd.tools.plotting.table)
# FIXME index column toooo wide
fig, ax = plt.subplots(1,1)
fig.suptitle('Statistics on the absolute value of correlation coefficients')
ax.axis('off')
ax.set_position([.5, 0.2, 0.5, 0.6])
a = df_epoch(np.round(np.abs(ds).describe(),4).T)
cw = np.ones((len(a.columns),))
t = pd.tools.plotting.table(ax, a, loc='upper right', fontsize=12, colWidths=cw/np.sum(cw))
pp.savefig()
plt.close(fig)
a

In [ ]:
change = np.zeros((num_plots,num_plots))
for idx1 in range(0,num_plots):
    for idx2 in range(0,num_plots):
        change[idx1,idx2] = np.linalg.norm(np.ravel(mx[et.index[idx1]]-mx[et.index[idx2]])/np.size(mx[et.index[idx2]]),2)


fig, ax = plt.subplots(1,1, figsize=(6,6))
fig.tight_layout(rect=[0.4,0,0.95,0.55])
#fig = plt.figure()
#ax = fig.gca()
cax = ax.matshow(change+np.diag(np.nan*np.diag(change)), cmap=plt.get_cmap('rainbow'))

fig.suptitle('Difference between test cases (RMS distance)\n'+', '.join(et.index.names))
ax.set_xticks(np.array(range(0,len(et.index))))
ax.set_xticklabels(et.index.values.tolist(),rotation=90)
ax.set_yticks(np.array(range(0,len(et.index))))
ax.set_yticklabels(et.index.values.tolist())

# without set_yticks
# ax.set_yticklabels([tuple()]+et.index.values.tolist())
fig.colorbar(cax)
pp.savefig()

In [ ]:
pp.close()

### An example of spiking
The first 1 second of the recording seems missing

In [ ]:
def plot_firing(ax, experiment_id, rois=None):
    # Plot all neural units in this experiment
    if rois is None:
        rois = transients_data.loc[experiment_id].index.unique()
    for i in range(0,len(rois)):
        unit = rois[i]
        try:
            firing = np.array(transients_data.loc[(experiment_id,unit),['start_frame', 'stop_frame']])
            if len(firing):
                ax.plot(firing.T,i*np.ones_like(firing.T),c='k')
            firing = transients_data.loc[(experiment_id,unit),'max_frame']
            if len(firing):
                ax.plot(firing,i*np.ones_like(firing),'|',ms=5)
        except:
            pass
    for i in range(0,len(events)):
        ax.axvline(x=events[i]*FPS, ymin=0.0, ymax = 1.0, linewidth=1, color='k')
    ax.set_title('Transient peaks and durations ExID: '+experiment_id)
    ax.set_xlabel('Camera frame')
    ax.set_ylabel('Unit ID')

In [ ]:
et3 = experiment_traits.copy().reset_index(drop=True)
et3.loc[:,'session_num'] = et3.loc[:,'session_num'].astype(int)
et3 = et3.sort_values(['learning_epoch','context','port','puffed','session_num']).set_index(['learning_epoch','context','port','puffed'])

In [ ]:
pp = helpmultipage(animal+'_firing.pdf')

In [ ]:
xmax = transients_data.loc[:,['stop_frame']].max().values

for idx, val in et3.iterrows():
    fig, ax = plt.subplots(1,1,figsize=(16,10))
    ax.set_xlim(xmax=xmax)
    experiment_id = val['time']
    print (experiment_id)
    fig.suptitle('learning_epoch, context, port, puffed: #context in epoch\n'+
        '%s: session %s'%(idx,val['session_num']))
    plot_firing(ax, experiment_id, rois.values)
    pp.savefig()
    plt.close(fig)

In [ ]:
pp.close()